In [1]:
from vnstock import *
from indices_helper import *
import importlib
import indices_helper

importlib.reload(indices_helper)

df_data = get_stock_price('VHM', 180)

In [2]:
import pandas as pd
import plotly.graph_objects as go

# Function to calculate Ichimoku Cloud values
def calculate_ichimoku_cloud(df):
    # Define parameters
    period9_high = df['high'].rolling(window=9).max()
    period9_low = df['low'].rolling(window=9).min()

    period26_high = df['high'].rolling(window=26).max()
    period26_low = df['low'].rolling(window=26).min()

    tenkan_sen = (period9_high + period9_low) / 2
    kijun_sen = (period26_high + period26_low) / 2

    senkou_span_a = ((tenkan_sen + kijun_sen) / 2).shift(26)

    period52_high = df['high'].rolling(window=52).max()
    period52_low = df['low'].rolling(window=52).min()

    senkou_span_b = ((period52_high + period52_low) / 2).shift(26)

    return tenkan_sen, kijun_sen, senkou_span_a, senkou_span_b

# Function to plot Ichimoku Cloud
def plot_ichimoku_cloud(df, buy_signals=None, sell_signals=None):
    tenkan_sen, kijun_sen, senkou_span_a, senkou_span_b = calculate_ichimoku_cloud(df)

    # Create figure and add traces
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df.index, y=df['close'], mode='lines', name='Close', line=dict(color='black')))
    fig.add_trace(go.Scatter(x=df.index, y=tenkan_sen, mode='lines', name='Tenkan-sen', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=df.index, y=kijun_sen, mode='lines', name='Kijun-sen', line=dict(color='red')))
    fig.add_trace(go.Scatter(x=df.index, y=senkou_span_a, fill='tonexty', mode='lines', name='Senkou Span A',
                             line=dict(color='lightgreen')))
    fig.add_trace(go.Scatter(x=df.index, y=senkou_span_b, fill='tonexty', mode='lines', name='Senkou Span B',
                             line=dict(color='lightcoral')))

    if buy_signals is not None:
        fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['close'], mode='markers', name='Buy Signal',
                                 marker=dict(color='green', size=8)))

    if sell_signals is not None:
        fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['close'], mode='markers', name='Sell Signal',
                                 marker=dict(color='red', size=8)))

    fig.update_layout(title='Ichimoku Cloud Trading Strategy', xaxis_title='Date', yaxis_title='Price')
    fig.show()



# Plotting the Ichimoku Cloud
plot_ichimoku_cloud(df_data)
